<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#form https://docs.google.com/forms/d/e/1FAIpQLSdH_5zoN0BvuYJdUaIYjQDkz7niBVXR671zpUm3p-RqJdBDfA/viewform

#handout https://www.depends-on-the-definition.com/lstm-with-char-embeddings-for-ner/

#lect notebook https://colab.research.google.com/drive/1xb3OD0b8IPqqyVUqRtYu6NgE27Brgc_P?usp=sharing


wtf


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, InputLayer, Embedding, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

In [0]:
from numpy.random import seed
seed(0)
from tensorflow.random import set_seed
set_seed(0)

In [5]:
!wget https://raw.githubusercontent.com/divamgupta/lstm-gender-predictor/master/female.txt
!wget https://raw.githubusercontent.com/divamgupta/lstm-gender-predictor/master/male.txt

--2020-05-19 15:47:02--  https://raw.githubusercontent.com/divamgupta/lstm-gender-predictor/master/female.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35575 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.74K  --.-KB/s    in 0.01s   

2020-05-19 15:47:03 (2.43 MB/s) - ‘female.txt’ saved [35575/35575]

--2020-05-19 15:47:05--  https://raw.githubusercontent.com/divamgupta/lstm-gender-predictor/master/male.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20292 (20K) [text/plain]
Saving t

In [0]:
with open("male.txt") as f:
    m_names = f.readlines()

with open("female.txt") as f:
    f_names = f.readlines()

In [7]:
print(f'{len(f_names)} женских имен, {len(m_names)} мужских имен')

5001 женских имен, 2943 мужских имен


In [8]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [0]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [0]:
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(sentence)
    sentence_tags.append(tags)

In [0]:
sent_train, sent_test, tag_train, tag_test = train_test_split(sentences, sentence_tags, test_size=0.2, random_state=0)

In [0]:
vocab = Counter()
for sent in sent_train:
    sent = [word.lower() for word in sent]
    vocab.update(sent)

In [0]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}

In [0]:
word2id = {'PAD':0,'UNK':1}    
for i,word in enumerate(filtered_vocab):
      word2id[word] = i + 2

id2word = {i:word for word, i in word2id.items()}

In [0]:
tag2id = {'PAD':0}  
for tags in tag_train:
    for tag in tags:
      if tag.lower() not in tag2id:
        tag2id[tag.lower()] = len(tag2id)

id2tag = {i:tag for tag, i in tag2id.items()}

In [0]:
def data2ints(data, smth2id):
  int_data = []
  for seq in data:
      int_seq = []
      for i in seq:
          try:
            int_seq.append(smth2id[i.lower()])
          except KeyError:
            int_seq.append(smth2id['UNK'])
  
      int_data.append(int_seq)
  return int_data
 

In [18]:
X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
y_train_ids, y_test_ids = data2ints(tag_train, tag2id), data2ints(tag_test, tag2id)


print(X_train_ids[0])
print(X_test_ids[0])
print(y_train_ids[0])
print(y_test_ids[0])

[673, 1, 1, 1, 323, 1, 794, 423, 786]
[481, 1460, 656, 628, 432, 1652, 56, 1, 27, 1, 471, 280, 1238, 786]
[1, 1, 1, 2, 1, 1, 3, 4, 5]
[18, 19, 21, 24, 10, 25, 24, 18, 21, 14, 3, 7, 15, 5]


In [0]:
MAX_LEN = max(len(x) for x in X_train_ids)

In [23]:
MAX_LEN

128

In [0]:
X_train, X_test = pad_sequences(X_train_ids, maxlen=MAX_LEN, padding='post'), pad_sequences(X_test_ids, maxlen=MAX_LEN, padding='post')
y_train_pad, y_test_pad = pad_sequences(y_train_ids, maxlen=MAX_LEN, padding='post'), pad_sequences(y_test_ids, maxlen=MAX_LEN, padding='post')

In [28]:
print(X_train.shape, y_train_pad.shape, X_test.shape, y_test_pad.shape)

(3131, 128) (3131, 128) (783, 128) (783, 128)


In [0]:
y_train, y_test = to_categorical(y_train_pad, num_classes=len(tag2id)), to_categorical(y_test_pad, num_classes=len(tag2id))

In [30]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LEN)))
model.add(Embedding(len(word2id),100))
model.add(Bidirectional(LSTM(16, return_sequences=True), merge_mode='concat'))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(tag2id))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 100)          168100    
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 32)           14976     
_________________________________________________________________
dropout (Dropout)            (None, 128, 32)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 47)           1551      
_________________________________________________________________
activation (Activation)      (None, 128, 47)           0         
Total params: 184,627
Trainable params: 184,627
Non-trainable params: 0
_________________________________________________________________
